   ![Texte alternatif](image.png)

In [24]:
from colorthief import ColorThief

color_thief = ColorThief("image.png")
palette = color_thief.get_palette(color_count=4)

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

plotly_colors = [rgb_to_hex(c) for c in palette]

# HEART RISK Data Visualization

Gauthier Meffe, Noa Chaze, Thomas Dos, Lisa Escaron

In [1]:
import pandas as pd 
import numpy as np 

In [3]:
df_train = pd.read_csv('data/train.csv')

In [4]:
df_test = pd.read_csv('data/test.csv')

## Description du jeu de données

Lors d'un appel téléphonique, 325 questions sont posées (une pour chaque colonne de notre jeu de données). 

La première information donnée (première colonne STATE) est l'état de résidence de la personne (état vers lequel l'appel est envoyé). On peut ainsi visualiser sur la carte des Etat-Unis la provenance des participants.

In [33]:
import plotly.express as px

# Dictionnaire de correspondance FIPS → code état à deux lettres
fips_to_state = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT',
    '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL',
    '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
    '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE',
    '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND',
    '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD',
    '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
    '55': 'WI', '56': 'WY'
}

# Compter le nombre de personnes par état
state_counts = df_train['_STATE'].value_counts().reset_index()
state_counts.columns = ['_STATE', 'count']

# Convertir FIPS en string à 2 chiffres puis en code état à 2 lettres
state_counts['_STATE'] = state_counts['_STATE'].astype(int).astype(str).str.zfill(2)
state_counts['state_code'] = state_counts['_STATE'].map(fips_to_state)

# Retirer les lignes sans correspondance (ex: territoires hors USA)
state_counts = state_counts.dropna(subset=['state_code'])

custom_gradient = ["white", plotly_colors[0]]

fig = px.choropleth(
    state_counts,
    locations='state_code',
    locationmode="USA-states",
    color='count',
    scope="usa",
    color_continuous_scale=custom_gradient,
    title="Nombre de personnes appelées par état"
)
fig.show()

Ces appels sont passés pour évaluer les causes possibles de risque cardiaque chez une personne. Ainsi la dernière question posée concerne les antécédents de crise cardiaqe. Sur les personnes appelées, la répartition des personnes ayant eu une crise cardiaque est la suivante :

In [34]:
import plotly.graph_objects as go

# Calcul du pourcentage pour chaque classe de TARGET
target_counts = df_train['TARGET'].value_counts(normalize=True)
labels = ['Arrêt cardiaque', 'Aucun arrêt cardiaque']
values = [target_counts[True]*100, target_counts[False]*100]


colors = [plotly_colors[0], plotly_colors[1]]

fig_target = go.Figure(data=[
    go.Bar(
        x=labels,
        y=values,
        marker_color=colors,
        text=[f"{v:.1f}%" for v in values],
        textposition='auto'
    )
])

fig_target.update_layout(
    title="Répartition des personnes ayant déjà eu un arrêt cardiaque sur le panel des personnes appelées",
    yaxis_title="Pourcentage (%)",
    xaxis_title="",
    plot_bgcolor='white'
)
fig_target.show()

On peut alors visualiser sur la carte des Etats-Unis le nombre de personnes ayant eu une crise cardiaque par état de résidence : 

In [36]:
# Calculer le nombre de personnes ayant eu un arrêt cardiaque par état
state_target_counts = df_train[df_train['TARGET'] == True]['_STATE'].value_counts().reset_index()
state_target_counts.columns = ['_STATE', 'cardiac_arrest_count']

# Convertir FIPS en string à 2 chiffres puis en code état à 2 lettres
state_target_counts['_STATE'] = state_target_counts['_STATE'].astype(int).astype(str).str.zfill(2)
state_target_counts['state_code'] = state_target_counts['_STATE'].map(fips_to_state)

# Retirer les lignes sans correspondance (ex: territoires hors USA)
state_target_counts = state_target_counts.dropna(subset=['state_code'])


fig_cardiac = px.choropleth(
    state_target_counts,
    locations='state_code',
    locationmode="USA-states",
    color='cardiac_arrest_count',
    scope="usa",
    color_continuous_scale=custom_gradient,
    title="Nombre de personnes ayant eu un arrêt cardiaque par état"
)
fig_cardiac.show()

Cette carte n'a pas beaucoup de sens car on a vu précédemment qu'il n'y a pas eu le même nombre de personne intérrogées par état. On peut donc normaliser le nombre de personnes ayant eu une crise cardiaque par le nombre total de personnes appelées dans chaque état. On obtient alors la carte suivante :

In [38]:
# Calculer le taux de crise cardiaque par état (nombre de cas / nombre d'appels)
merged = state_counts.merge(state_target_counts, on='state_code', how='left')
merged['cardiac_arrest_count'] = merged['cardiac_arrest_count'].fillna(0)
merged['cardiac_arrest_rate'] = merged['cardiac_arrest_count'] / merged['count']

fig_rate = px.choropleth(
    merged,
    locations='state_code',
    locationmode="USA-states",
    color='cardiac_arrest_rate',
    scope="usa",
    color_continuous_scale=custom_gradient,
    title="Taux de crise cardiaque par état (parmi les personnes appelées)"
)
fig_rate.show()

Maintenant qu'on a vu géographiquement d'où venait les personnes appelées et d'où venait les personnes ayant eu déjà une crise cardiaque, on peut s'intéresser aux autres questions posées lors de l'appel téléphonique. Parmi les 325 questions posées, on en a sélectionné quelques unes qui nous semblaient intéressantes à analyser.

#### Profil des personnes appelées

On peut visualiser l'âge et le sexe des personnes appelées. Toutes les personnes au dessus de 80 ans sont regroupées dans la même catégorie, on supprime alors ce groupe pour avoir une meilleure visualisation de tous les âges en dessous. 

In [46]:
import plotly.express as px

# Supposons que 'SEXVAR' (1=Male, 2=Female) et '_AGEG5YR' (groupe d'âge) sont les colonnes d'intérêt
age_sex_counts = df_train.groupby(['_AGE80', 'SEXVAR']).size().reset_index(name='count')

# Mapping pour les labels de sexe
sex_labels = {1: 'Homme', 2: 'Femme'}
age_sex_counts['Sexe'] = age_sex_counts['SEXVAR'].map(sex_labels)

age_sex_counts_filtered = age_sex_counts[age_sex_counts['_AGE80'] < 80]
fig_age_sex = px.bar(
    age_sex_counts_filtered,
    x='_AGE80',
    y='count',
    color='Sexe',
    barmode='overlay',
    opacity=0.6,
    color_discrete_sequence=plotly_colors[:2],
    labels={'_AGE80': "Âges", 'count': 'Nombre de personnes'},
    title="Nombre de personnes appelées par groupe d'âge et par sexe (hors 80+)"
)
fig_age_sex.show()

### 1. Données manquantes : 

In [9]:
for i in df_train.columns:
    null_rate = df_train[i].isna().sum() / len(df_train) * 100 
    if null_rate > 0 :
        print("{} null rate: {}%".format(i,round(null_rate,2)))

CTELENM1 null rate: 78.54%
PVTRESD1 null rate: 78.54%
COLGHOUS null rate: 99.99%
STATERE1 null rate: 78.54%
CELPHON1 null rate: 78.54%
LADULT1 null rate: 78.54%
COLGSEX1 null rate: 99.99%
NUMADULT null rate: 78.54%
LANDSEX1 null rate: 90.07%
NUMMEN null rate: 89.1%
NUMWOMEN null rate: 89.1%
RESPSLCT null rate: 88.5%
SAFETIME null rate: 21.46%
CTELNUM1 null rate: 21.46%
CELLFON5 null rate: 21.46%
CADULT1 null rate: 21.46%
CELLSEX1 null rate: 21.46%
PVTRESD3 null rate: 21.47%
CCLGHOUS null rate: 99.67%
CSTATE1 null rate: 21.47%
LANDLINE null rate: 21.47%
HHADULT null rate: 21.47%
GENHLTH null rate: 0.0%
PHYSHLTH null rate: 0.0%
MENTHLTH null rate: 0.0%
POORHLTH null rate: 42.83%
PRIMINSR null rate: 0.0%
PERSDOC3 null rate: 0.0%
MEDCOST1 null rate: 0.0%
CHECKUP1 null rate: 0.0%
EXERANY2 null rate: 0.0%
SLEPTIM1 null rate: 0.0%
LASTDEN4 null rate: 0.45%
RMVTETH4 null rate: 0.45%
CVDSTRK3 null rate: 0.0%
ASTHMA3 null rate: 0.0%
ASTHNOW null rate: 85.28%
CHCSCNC1 null rate: 0.0%
CHCOCNC1 nul

La plupart de colonnes ont des données manquantes : les participants ne sont pas forcés de répondre à toutes les questions ou n'ont pas forcément toutes les informations demandées. Ainsi notre jeu de données est très hétérogène, avec des colonnes ayant jusqu'à 99% de données manquantes.

### 2. Sélection de variables : 

Pour l'étude de prédiction, nous avons parcouru manuellement la liste des questions posées pour supprimer les questions qui ne nous semblaient pas pertienentes pour la prédiction du risque cardiaque (comme la date de l'appel ou l'état de résidence).

In [10]:
df_train = df_train.drop(columns=[
    'FMONTH', 'IDATE', 'IMONTH', 'IYEAR', 'SEQNO', '_PSU', 'CTELENM1', 
    'PVTRESD1', 'COLGHOUS', 'STATERE1', 'CELPHON1', 'LADULT1', 'COLGSEX1', 
    'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'RESPSLCT', 'LANDSEX1', 'CELLSEX1', 
    'SAFETIME', 'CTELNUM1', 'CELLFON5', 'CADULT1', 'PVTRESD3', 'CCLGHOUS', 
    'CSTATE1', 'LANDLINE', 'HHADULT', 'RMVTETH4', 'NUMHHOL4', 
    'VIRCOLO1', 'TOLDCFS', 'HAVECFS', 'WORKCFS', 'CAREGIV1', 'CRGVREL4', 
    'CSRVSUM', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB3', 'CRGVALZD', 'CRGVPER1', 
    'CRGVHOU1', 'CRGVEXPT', 'FIREARM5', 'GUNLOAD', 'LOADULK2', 'RCSGEND1', 
    'RCSXBRTH', 'RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 'BIRTHSEX', 'WHEREGET', 
    'NOBCUSE8', 'RRCOGNT2', 'QSTVER', 'QSTLANG', '_STSTR', '_STRWT', 
    '_RAWRAKE', '_WT2RAKE', 'CAGEG', '_CLLCPWT', '_DUALUSE', '_DUALCOR', 
    '_LLCPWT2', '_LLCPWT', '_EXTETH3', '_ALTETH3', '_PRACE2'
])

df_test = df_test.drop(columns=[
    'FMONTH', 'IDATE', 'IMONTH', 'IYEAR', 'SEQNO', '_PSU', 'CTELENM1', 
    'PVTRESD1', 'COLGHOUS', 'STATERE1', 'CELPHON1', 'LADULT1', 'COLGSEX1', 
    'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'RESPSLCT', 'LANDSEX1', 'CELLSEX1', 
    'SAFETIME', 'CTELNUM1', 'CELLFON5', 'CADULT1', 'PVTRESD3', 'CCLGHOUS', 
    'CSTATE1', 'LANDLINE', 'HHADULT', 'RMVTETH4', 'NUMHHOL4', 
    'VIRCOLO1', 'TOLDCFS', 'HAVECFS', 'WORKCFS', 'CAREGIV1', 'CRGVREL4', 
    'CSRVSUM', 'CRGVLNG1', 'CRGVHRS1', 'CRGVPRB3', 'CRGVALZD', 'CRGVPER1', 
    'CRGVHOU1', 'CRGVEXPT', 'FIREARM5', 'GUNLOAD', 'LOADULK2', 'RCSGEND1', 
    'RCSXBRTH', 'RCSRLTN2', 'CASTHDX2', 'CASTHNO2', 'BIRTHSEX', 'WHEREGET', 
    'NOBCUSE8', 'RRCOGNT2', 'QSTVER', 'QSTLANG', '_STSTR', '_STRWT', 
    '_RAWRAKE', '_WT2RAKE', 'CAGEG', '_CLLCPWT', '_DUALUSE', '_DUALCOR', 
    '_LLCPWT2', '_LLCPWT', '_EXTETH3', '_ALTETH3', '_PRACE2'
])

On supprime aussi les colonnes avec plus de 60% de données manquantes : 

In [11]:
missing_rates = df_train.isna().mean()
cols_to_drop = missing_rates[missing_rates > 0.6].index.tolist()

# Suppression de ces colonnes dans les deux jeux de données
df_train = df_train.drop(columns=cols_to_drop)
df_test = df_test.drop(columns=cols_to_drop)

print(f"Colonnes supprimées ({len(cols_to_drop)}):", cols_to_drop)

Colonnes supprimées (126): ['ASTHNOW', 'DIABAGE4', 'NUMPHON4', 'PREGNANT', 'HOWLONG', 'CRVCLCNC', 'CRVCLPAP', 'CRVCLHPV', 'SIGMTES1', 'LASTSIG4', 'VCLNTES2', 'SMALSTOL', 'STOLTEST', 'STOOLDN2', 'BLDSTFIT', 'SDNATES1', 'SMOKDAY2', 'LCSFIRST', 'LCSLAST', 'LCSNUMCG', 'LCSSCNCR', 'LCSCTWHN', 'HIVTSTD3', 'COVIDSMP', 'COVIDPRM', 'PDIABTS1', 'PREDIAB2', 'DIABTYPE', 'INSULIN1', 'CHKHEMO3', 'EYEEXAM1', 'DIABEYE1', 'DIABEDU1', 'FEETSORE', 'IMFVPLA3', 'HPVADVC4', 'HPVADSHT', 'SHINGLE2', 'COVIDVA1', 'COVIDNU1', 'COVIDFS1', 'COVIDSE1', 'COPDCOGH', 'COPDFLEM', 'COPDBRTH', 'COPDBTST', 'COPDSMOK', 'CNCRDIFF', 'CNCRAGE', 'CNCRTYP2', 'CSRVTRT3', 'CSRVDOC1', 'CSRVRTRN', 'CSRVINST', 'CSRVINSR', 'CSRVDEIN', 'CSRVCLIN', 'CSRVPAIN', 'CSRVCTL2', 'PSATEST1', 'PSATIME1', 'PCPSARS2', 'PSASUGST', 'PCSTALK1', 'CIMEMLOS', 'CDHOUSE', 'CDASSIST', 'CDHELP', 'CDSOCIAL', 'CDDISCUS', 'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN', 'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM', 'ACEHVSEX', 'ACEADS